In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
np.random.seed(42)
# github practice

In [ ]:
train=pd.read_csv("train.csv",header=0,index_col="Id")

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.dropna(thresh=len(train) - 460, axis=1,inplace=True)
train.info()

In [ ]:
float_cols = train.select_dtypes(include=['float64']).columns
str_cols = train.select_dtypes(include=['object']).columns
int_cols=train.select_dtypes(include=['int64']).columns
train.loc[:, float_cols] = train.loc[:, float_cols].fillna(train.median())
#train.loc[:, str_cols] = train.loc[:, str_cols].fillna('')
train.loc[:, int_cols] = train.loc[:, int_cols].fillna(train.median())
train=train.dropna()

In [ ]:
train.info()

In [ ]:
test=pd.read_csv("test.csv",header=0,index_col="Id")
float_cols = test.select_dtypes(include=['float64']).columns
str_cols = test.select_dtypes(include=['object']).columns
int_cols=test.select_dtypes(include=['int64']).columns
test.loc[:, float_cols] = test.loc[:, float_cols].fillna(test.median())
#train.loc[:, str_cols] = train.loc[:, str_cols].fillna('')
test.loc[:, int_cols] = test.loc[:, int_cols].fillna(test.median())

In [ ]:
test.dropna(thresh=len(test) - 460, axis=1,inplace=True)
test = test.fillna(test.mode().iloc[0])
test.info()

In [ ]:
X_train_df=train.drop('SalePrice', axis=1)
X_train_df= pd.get_dummies(X_train_df, prefix_sep="_",columns=X_train_df.select_dtypes(include=['object']).columns)
x_test_df=test
x_test_df= pd.get_dummies(x_test_df, prefix_sep="_",columns=x_test_df.select_dtypes(include=['object']).columns)
#this cell does not work with the code below. try to find different columns code.
#also, you can fill nans of the test data.

different_columns_train=X_train_df.columns.difference(x_test_df.columns)
different_columns_train=list(different_columns_train)

different_columns_test=x_test_df.columns.difference(X_train_df.columns)
different_columns_test=list(different_columns_test)

print("Columns only X_train_df has: ", different_columns_train)
print("Columns only X_test_df has: ", different_columns_test)


In [ ]:
for i in different_columns_train:
    x_test_df[i]=0

for i in different_columns_test:
    X_train_df[i]=0
#set(X_train_df.columns).symmetric_difference(set(x_test_df.columns))
different_columns_check=X_train_df.columns.difference(x_test_df.columns)
different_columns_check

In [ ]:

X_train_values= X_train_df.values

y_train=np.log(train['SalePrice']) 


In [ ]:
n_cols=X_train_values[1].shape
n_cols

In [ ]:
model=Sequential()
model.add(Dense(128, activation='relu', input_shape=n_cols ))
model.add(Dense(128, activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(1))

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping_monitor=EarlyStopping(patience=10)
from keras import optimizers
opt=optimizers.Adam(lr=0.01)
#from keras import backend as K
#def root_mean_squared_error(y_true, y_pred):
        #return K.sqrt(K.mean(K.square(y_pred - y_true)))
model.compile(optimizer=opt, loss = "mean_squared_error",metrics=['mse'])
#from sklearn import preprocessing
#X = preprocessing.normalize(X)



In [ ]:
model.fit(X_train_values, y_train,validation_split=0.2,batch_size=1024,callbacks=[early_stopping_monitor], epochs=100,verbose=1)


In [ ]:
x_test_values=x_test_df.values
#x_test_normalized = preprocessing.normalize(x_test_values)

y_predicted = model.predict(x_test_values)


In [ ]:
y_predicted=np.exp(y_predicted)

In [ ]:
y_predicted

In [ ]:
len(y_predicted)-np.count_nonzero(~np.isnan(y_predicted)) #check if it is there any nan predicted

In [ ]:
results = pd.DataFrame(data=y_predicted, index=x_test_df.index, columns=["SalePrice"])
results.head()

In [ ]:
results.reset_index(level=0, inplace=True)
results.head(15)

In [ ]:
results.tail(15)

In [ ]:
results.to_csv("submission_15_05_v03",index=False)